### Changing the objective functionprint(model.objective)

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from cobra.util import create_stoichiometric_matrix
from cobra.medium import minimal_medium
import numpy as np
import escher

In [67]:
model = read_sbml_model('iMM904.xml') # insert model HERE

# Basic initiation

You can change the objective function like this (e.g. if you'd like to determine the maximum possible production rate of acetate).

In [4]:
print(model.objective)

Maximize
1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090


We can get the $\mu_{max}$ before making any alterations to the model.

In [5]:
model.optimize().objective_value

0.28786570370401743

# The composition of the medium

Let's look at the default media composition

In [6]:
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

text text text text text

In [9]:
medium = model.medium
model.add_boundary(model.metabolites.pheme_m, type='demand') # adds the exchange reaction for heme
with model:
    solution = model.optimize()
    print("Standard yeast maximum theoretical biomass productivity:", solution.fluxes['BIOMASS_SC5_notrace'], '/h')
    model.medium = medium
    model.objective = model.reactions.DM_pheme_m
    heme_production = model.optimize().objective_value
    print("Standard yeast maximum theoretical productivity of heme ", heme_production, '[mmol gDW^-1 h^-1]')

Standard yeast maximum theoretical biomass productivity: 0.28786570370403197 /h
Standard yeast maximum theoretical productivity of heme  0.550189393939394 [mmol gDW^-1 h^-1]


We'll need to change the composition to simulate Yeast Nitrogen Base without amino acids


Most of the experimental data we are going to compare this GSMM with is based on cultivations of CEN.PK 113-7D in YNB media. Therefore, we will have to change the media.
https://www.carlroth.com/com/en/nutrient-media--molecular-biology/yeast-nitrogen-base-%28without-amino-acids%29/p/hp26.1


In [7]:
model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

In [58]:
def ynb(model):
    YNBmedium = model.medium
    ls=[]
    for i in range (0,len(list(YNBmedium))):
        YNBmedium[list(medium)[i]] = 0
        
    # Ammonium sulphate 5,0 g/l (NH₄)₂SO₄
    YNBmedium['EX_nh4_e'] += (5/132.14)*2
    YNBmedium['EX_so4_e'] += (5/132.14)
    
    # Potassium phosphate 1,0 g/l KH2PO4
    YNBmedium['EX_k_e'] += (1/136.086)
    YNBmedium['EX_h_e'] += (1/136.086)*2
    YNBmedium['EX_pi_e'] += (1/136.086)
    YNBmedium['EX_o2_e'] += (1/136.086)*4
    
    return YNBmedium

print(ynb(model))

{'EX_fe2_e': 0, 'EX_glc__D_e': 0, 'EX_h2o_e': 0, 'EX_h_e': 0.01469658892171127, 'EX_k_e': 0.007348294460855635, 'EX_na1_e': 0, 'EX_so4_e': 0.03783865597093992, 'EX_nh4_e': 0.07567731194187984, 'EX_o2_e': 0.02939317784342254, 'EX_pi_e': 0.007348294460855635}


In [71]:
max_growth = model.slim_optimize()
print(max_growth)
minimal_medium(model, max_growth)

0.287865703704017


EX_glc__D_e    10.000000
EX_so4_e        0.022252
EX_nh4_e        1.611029
EX_o2_e         2.000000
EX_pi_e         0.056911
dtype: float64

In [ ]:
YNBmedium = model.medium # https://openwetware.org/wiki/Composition_of_Yeast_Nitrogen_Base_(YNB)
YNBmedium[''] = 20 # Ammonium sulphate 5,0 g/l (NH₄)₂SO₄
YNBmedium[''] = 10 # Potassium phosphate 1,0 g/l KH2PO4
YNBmedium[''] = 10 # Magnesium sulphate 0,5 g/l MgSO₄
YNBmedium[''] = 10 # Calcium chloride 0,1 g/l CaCl2
YNBmedium[''] = 10 # Sodium chloride (NaCl) 0,1 g/l NaCl

YNBmedium[''] = 10 # Inositol 2,0 mg/l
YNBmedium[''] = 10 # Boric acid 0,5 mg/l
YNBmedium[''] = 10 # Niacin 0,4 mg/l
YNBmedium[''] = 10 # Thiamine hydrochloride 0,4 mg/l
YNBmedium[''] = 10 # Pyridoxin hydrochloride 0,4 mg/l
YNBmedium[''] = 10 # Calcium pantothenate 0,4 mg/l
YNBmedium[''] = 10 # Manganese sulphate 0,4 mg/l
YNBmedium[''] = 10 # Zinc sulphate 0,4 mg/l
YNBmedium[''] = 10 # p-Aminobenzoic acid 0,2 mg/l
YNBmedium[''] = 10 # Iron chloride 0,2 mg/l
YNBmedium[''] = 10 # Riboflavine 0,2 mg/l
YNBmedium[''] = 10 # Sodium molybdate 0,2 mg/l
YNBmedium[''] = 10 # Potassium iodide 0,1 mg/l
YNBmedium[''] = 10 # Biotin 0,002 mg/l
YNBmedium[''] = 10 # Folic acid 0,002 mg/l

In [44]:
with model:
    
    solution = model.optimize()
    print("With default medium:")
    print("Standard yeast maximum theoretical biomass productivity:", str(solution.fluxes['BIOMASS_SC5_notrace'])[:5], '/h')
    model.objective = model.reactions.DM_pheme_m
    heme_production = model.optimize().objective_value
    print("Standard yeast maximum theoretical productivity of heme ", str(heme_production)[:5], '[mmol gDW^-1 h^-1]')
    print(model.medium)
    
    for i in range (0,len(list(medium))):
        medium[list(medium)[i]] = 2
    model.medium = medium
    model.objective = model.reactions.DM_pheme_m
    heme_production = model.optimize().objective_value
    
    print("")
    print("Standard yeast maximum theoretical biomass productivity:", str(solution.fluxes['BIOMASS_SC5_notrace'])[:5], '/h')
    print("Standard yeast maximum theoretical productivity of heme ", str(heme_production)[:5], '[mmol gDW^-1 h^-1]')
    print(medium)

With default medium:
Standard yeast maximum theoretical biomass productivity: 0.287 /h
Standard yeast maximum theoretical productivity of heme  0.550 [mmol gDW^-1 h^-1]
{'EX_fe2_e': 999999.0, 'EX_glc__D_e': 10.0, 'EX_h2o_e': 999999.0, 'EX_h_e': 999999.0, 'EX_k_e': 999999.0, 'EX_na1_e': 999999.0, 'EX_so4_e': 999999.0, 'EX_nh4_e': 999999.0, 'EX_o2_e': 2.0, 'EX_pi_e': 999999.0}

Standard yeast maximum theoretical biomass productivity: 0.287 /h
Standard yeast maximum theoretical productivity of heme  0.240 [mmol gDW^-1 h^-1]
{'EX_fe2_e': 2, 'EX_glc__D_e': 2, 'EX_h2o_e': 2, 'EX_h_e': 2, 'EX_k_e': 2, 'EX_na1_e': 2, 'EX_so4_e': 2, 'EX_nh4_e': 2, 'EX_o2_e': 2, 'EX_pi_e': 2}


In [18]:
model.medium = YNBmedium
solution = model.optimize()
print("Standard yeast maximum theoretical biomass productivity:", solution.fluxes['BIOMASS_SC5_notrace'], '/h')
model.objective = model.reactions.DM_pheme_m
heme_production = model.optimize().objective_value
print("Standard yeast maximum theoretical productivity of heme ", heme_production, '[mmol gDW^-1 h^-1]')

KeyError: ''